In [ ]:
#Import libreries to use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Load Dataset Daily Group Consumption
daily_consumption_category = pd.read_csv('daily_consumption_category.csv')  # Category dataset
daily_consumption_group = pd.read_csv('daily_consumption_group.csv')        # Group dataset
daily_consumption_category[['day_avg_consumption','hour_avg_consumption']].astype(float) 
daily_consumption_group[['day_avg_consumption','hour_avg_consumption']].astype(float)
daily_consumption_category =np.round(daily_consumption_category,decimals=2)
daily_consumption_group = np.round(daily_consumption_group, 2)


### Formating Datasets

In [ ]:
daily_consumption_category=daily_consumption_category.sort_values(by='date_')
daily_consumption_group=daily_consumption_group.sort_values(by='date_')

In [ ]:
#Trasnfor data date values on format datetime
daily_consumption_category['date_'] = pd.to_datetime(daily_consumption_category['date_'])
daily_consumption_group['date_'] = pd.to_datetime(daily_consumption_group['date_'])

In [ ]:
#Weekday data generation
daily_consumption_category['weekday'] = pd.DatetimeIndex(daily_consumption_category['date_']).dayofweek
daily_consumption_group['weekday'] = pd.DatetimeIndex(daily_consumption_group['date_']).dayofweek

In [ ]:
#Weekday name generation category dataset
weekdaylist = [
    (daily_consumption_category['weekday'] == 0),
    (daily_consumption_category['weekday'] == 1),
    (daily_consumption_category['weekday'] == 2),
    (daily_consumption_category['weekday'] == 3),
    (daily_consumption_category['weekday'] == 4),
    (daily_consumption_category['weekday'] == 5),
    (daily_consumption_category['weekday'] == 6)]
wlist = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
daily_consumption_category["name_weekday"] = np.select(weekdaylist, wlist, default='Not Specified')

In [ ]:
#Weekday name generation group dataset
weekdaylist2 = [
    (daily_consumption_group['weekday'] == 0),
    (daily_consumption_group['weekday'] == 1),
    (daily_consumption_group['weekday'] == 2),
    (daily_consumption_group['weekday'] == 3),
    (daily_consumption_group['weekday'] == 4),
    (daily_consumption_group['weekday'] == 5),
    (daily_consumption_group['weekday'] == 6)]
wlist2 = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
daily_consumption_group["name_weekday"] = np.select(weekdaylist2, wlist2, default='Not Specified')

### Vizualization
Plots tab 1

In [ ]:
#Energy Consumption by Acorn Category
fig = px.line(daily_consumption_category,
              x="date_",
              y="day_avg_consumption",
              color='acorn_category',
              labels=dict(date_="Date", day_avg_consumption="Daily energy consume (kWh)", acorn_category='Category'),
              title="Energy Consumption By Acorn Category"
    )

fig.show()

In [ ]:
#Relation between average daily energy consumption and temperature per Acorn Category

#Load weather daily dataset
weather_daily = pd.read_csv('weather_daily_darksky.csv')

#Convert date in weather daily dataframe to day
weather_daily['date'] = pd.DatetimeIndex(weather_daily["time"]).to_period('D')

#Convert date in daily energy dataframe to day
daily_consumption_category['date'] = pd.DatetimeIndex(daily_consumption_category['date_']).to_period('D')

#Create subdataframe with daily energy consumption and weather variables
df1 = weather_daily.groupby('date')[['temperatureMin','cloudCover','uvIndex']].mean().reset_index()
df2 = daily_consumption_category.groupby(['date','acorn_category'])[['day_avg_consumption']].mean().reset_index()

#Merge daily energy consumption and weather dataset 
daily_with_weather = pd.merge(df2,df1,on = 'date',how = 'left')

fig = px.scatter(daily_with_weather,
                 x= daily_with_weather['temperatureMin'],
                 y= daily_with_weather['day_avg_consumption'],
                 color= daily_with_weather['acorn_category'],
                 title="Relation between average daily energy consumption and temperature per ACORN Category",
                 labels={'acorn_category':'Category', 'day_avg_consumption':"Average daily energy consume (kWh) per household",'temperatureMin':"Temperature (°C)"}
)
fig.show()

In [ ]:
#Categories Average

df = daily_consumption_category.sort_values('day_avg_consumption', ascending= False)

fig = px.bar(df, 
             x='acorn_category', 
             y="day_avg_consumption", 
             color='acorn_category',
             labels={'acorn_category':'Category', 'day_avg_consumption':'Mean daily energy consume (kWh)'},
             title="Categories Average",
             width=800,
             height=800
            )
fig.show()


  

In [ ]:
#Daily Relevance

cross_tab_prob = pd.crosstab(index=daily_consumption_category['weekday'], columns=daily_consumption_category['acorn_category'], normalize='index')
fig= px.bar(cross_tab_prob,
            title='Daily Relevance',
            labels={'value':'Proportion', 'weekday':'Weekday', 'acorn_category':'Category'},
            width=800,##size
            height=800
           )
fig.show()

Plots tab 2

In [ ]:
#Daily Behavior By Group

cross_tab_prob = pd.crosstab(index=daily_consumption_group['weekday'], columns=daily_consumption_group['acorn_group_detail'])
fig= px.bar(cross_tab_prob, title="Daily Behavior By Group",
            labels={'value':'Daily energy consume (Kw/h)','weekday':'Weekday','acorn_group_detail':'Group'},
            width=800,##size
            height=800)
fig.show()

In [ ]:
#Energy Consumption Behavior By Group
fig = px.line(daily_consumption_group,
              x="date_",
              y="day_avg_consumption",
              labels=dict(date_="Date", day_avg_consumption="Daily energy consume (kWh)",acorn_group_detail='Group'),
              hover_data={"date_": "|%B %d, %Y"},
              color='acorn_group_detail',
              title="Energy Consumption By Group"
    )

fig.show()


In [ ]:
#Relation between average daily energy consumption and temperature per Acorn Group

#Load weather daily dataset
weather_daily = pd.read_csv('weather_daily_darksky.csv')

#Convert date in weather daily dataframe to day
weather_daily['date'] = pd.DatetimeIndex(weather_daily["time"]).to_period('D')

#Convert date in daily energy dataframe to day
daily_consumption_group['date'] = pd.DatetimeIndex(daily_consumption_group['date_']).to_period('D')

#Create subdataframe with daily energy consumption and weather variables
df1 = weather_daily.groupby('date')[['temperatureMin','cloudCover','uvIndex']].mean().reset_index()
df2 = daily_consumption_group.groupby(['date','acorn_group_detail'])[['day_avg_consumption']].mean().reset_index()

#Merge daily energy consumption and weather dataset 
daily_with_weather = pd.merge(df2,df1,on = 'date',how = 'left')

fig = px.scatter(daily_with_weather,
                 x= daily_with_weather['temperatureMin'],
                 y= daily_with_weather['day_avg_consumption'],
                 color= daily_with_weather['acorn_group_detail'],
                 title="Relation between average daily energy consumption and temperature per Acorn Group",
                 labels={'acorn_group_detail':'Group', 'day_avg_consumption':"Average daily energy consume (kWh) per household",'temperatureMin':"Temperature (°C)"}
)
fig.show()